In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("marketing_campaign.csv", sep="\t")
df.head()


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [4]:
#brakujące dane

missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0] 

print(missing_values)

Income    24
dtype: int64


In [5]:
#Zamiana dat na ilość dni bycia klientem

df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%d-%m-%Y', errors='coerce')
reference_date = pd.to_datetime('08-02-2021', format='%d-%m-%Y', errors='coerce')
df['Dt_Customer'] = (reference_date - df['Dt_Customer']).dt.days

In [6]:
#odstające dane w Income
q1 = df['Income'].quantile(0.25)
q3 = df['Income'].quantile(0.75)

IQR = q3 - q1

outliers = df[(df['Income'] < (q1 - 1.5 * IQR)) | (df['Income'] > (q3 + 1.5 * IQR))]

outliers.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
164,8475,1973,PhD,Married,157243.0,0,1,2536,98,20,...,0,0,0,0,0,0,0,3,11,0
617,1503,1976,PhD,Together,162397.0,1,1,2807,31,85,...,1,0,0,0,0,0,0,3,11,0
655,5555,1975,Graduation,Divorced,153924.0,0,0,2558,81,1,...,0,0,0,0,0,0,0,3,11,0
687,1501,1982,PhD,Married,160803.0,0,0,3110,21,55,...,0,0,0,0,0,0,0,3,11,0
1300,5336,1971,Master,Together,157733.0,1,0,2806,37,39,...,1,0,0,0,0,0,0,3,11,0


In [73]:
#sprawdzanie wartości skrajnych

outliers_count = []

for col in df.columns[1:]:
    if df[col].dtype in ['int64', 'float64']:
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        IQR = q3 - q1
        outliers = df[(df[col] < (q1 - 1.5 * IQR)) | (df[col] > (q3 + 1.5 * IQR))]
        outliers_count.append(outliers.shape[0])
    else:
        outliers_count.append(0)

outliers_df = pd.DataFrame([outliers_count], columns=df.columns[1:])
outliers_df.index = ['Ilość wartości skrajnych']

outliers_df.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
Ilość wartości skrajnych,3,0,0,8,0,0,0,0,35,227,...,8,163,167,163,144,30,21,0,0,334


In [74]:
incl= ['mean','std']
num_cols = df.select_dtypes(include=['int', 'float']).drop(columns=['ID'])
statystyki = {
'Średnia':num_cols.mean(),
'Mediana':num_cols.median(),
'Minimum':num_cols.min(),
'Maksimum':num_cols.max(),
'Odchylenie Standardowe':num_cols.std(),
'Skośność':num_cols.skew()
}
#statystyki = pd.DataFrame(statystyki).transpose()


In [75]:
other_cols = [col for col in df.columns if col not in num_cols]

for i in statystyki:
 
    for k in other_cols:
       statystyki[i][k]='N/A'  

df['Education'].unique().tolist()

['Graduation', 'PhD', 'Master', 'Basic', '2n Cycle']

In [76]:

education_sort = {
    'Basic': 1,      
    '2n Cycle': 2,   
    'Graduation': 3, 
    'Master': 4,     
    'PhD': 5         
}
sorted_education = sorted(df['Education'].tolist(), key=lambda x: education_sort[x])

In [77]:
 
#mediana, min i max edukacji
if(len(sorted_education)%2==0):
    if(sorted_education[int(len(sorted_education)/2)]==sorted_education[int(len(sorted_education)/2+1)]):
        statystyki['Mediana']['Education']= sorted_education[int(len(sorted_education)/2)]
 

else:
    statystyki['Mediana']['Education']= sorted_education[int(len(sorted_education)/2)+1]
statystyki['Minimum']['Education']='Basic'
statystyki['Maksimum']['Education']='PhD'

In [78]:
statystyki = pd.DataFrame(statystyki).transpose()

statystyki = statystyki[df.columns.tolist()] 
statystyki.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
Średnia,N/A,1968.805804,N/A,N/A,52247.251354,0.444196,0.50625,2769.582143,49.109375,303.935714,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
Mediana,N/A,1970.0,Graduation,N/A,51381.5,0.0,0.0,2771.5,49.0,173.5,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0
Minimum,N/A,1893.0,Basic,N/A,1730.0,0.0,0.0,2416.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,11.0,0.0
Maksimum,N/A,1996.0,PhD,N/A,666666.0,2.0,2.0,3115.0,99.0,1493.0,...,20.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,11.0,1.0
Odchylenie Standardowe,N/A,11.984069,N/A,N/A,25173.076661,0.538398,0.544538,202.122512,28.962453,336.597393,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
